本demo以llvm为目标后端

In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing

在Relay中定义神经网络

+ 首先定义一个带有Relay Python前端的神经网络，简单期间，在Relay中使用预定义的resnet-18网络
+ 参数用Xavier初始化程序进行初始化，Relay还支持其他模型格式如MXNet、CoreML、ONNX和TensorFLow等

本教程假设在当前设备上进行推理，batch size为1，输入图像的大小为224 * 224的RGB彩色图像，调用`tvm.relay.expr.TupleWrapper.astext()`可以查看网络结构

In [3]:
batch_size = 1
num_class = 1000
image_shape = (3, 224, 224)
# 结果是(1, 3, 224, 224)
data_shape = (batch_size,) + image_shape
out_shape = (batch_size, num_class)

mod, params = relay.testing.resnet.get_workload(
    num_layers=18, batch_size=batch_size, image_shape=image_shape
)

# 想现实元数据则则设置show_meta_data=True
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %bn_data_gamma: Tensor[(3), float32] /* ty=Tensor[(3), float32] */, %bn_data_beta: Tensor[(3), float32] /* ty=Tensor[(3), float32] */, %bn_data_moving_mean: Tensor[(3), float32] /* ty=Tensor[(3), float32] */, %bn_data_moving_var: Tensor[(3), float32] /* ty=Tensor[(3), float32] */, %conv0_weight: Tensor[(64, 3, 7, 7), float32] /* ty=Tensor[(64, 3, 7, 7), float32] */, %bn0_gamma: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %bn0_beta: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %bn0_moving_mean: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %bn0_moving_var: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %stage1_unit1_bn1_gamma: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %stage1_unit1_bn1_beta: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %stage1_unit1_bn1_moving_mean: Tensor[(64), float32] /* ty=Tensor[(64), float32] */,

编译

用Relay/TVM管道编译模型，可以指定编译的优化级别（0-3），优化pass包括算子融合、预计算、布局变化等

`relay.build()`返回三个组件
+ JSON格式的执行图
+ 目标硬件上编译此执行图的函数组成的TVM模块库
+ 模型的blobs参数

两个部分优化的角度不同，一起来为模型服务提供优化的运行时模块，`relay.build()`首先进行一些图级别优化，例如剪枝、融合等，然后将算子（即优化图的节点）注册到TVM的实现，从而生成`tvm.module`，TVM进行张量级别的优化，然后，为了生成模块库，TVM首先将高级IR转换为指定目标后端的底层固有IR，然后生成的机器码将作为模块库
+ Relay进行图级别优化
+ TVM进行张量级别优化

In [ ]:
opt_level = 0
target = tvm.target.intel_graphics()
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)

运行生成库

In [ ]:
# create random input
dev = tvm.cpu()
data = np.random.uniform(-1, 1, size=data_shape).astype("float32")
# create module
module = graph_executor.GraphModule(lib["default"](dev))
# set input and parameters
module.set_input("data", data)
# run
module.run()
# get output
out = module.get_output(0, tvm.nd.empty(out_shape)).numpy()

# print first 10 elements of output
print(out.flatten()[0:10])

保存和加载编译模块

In [ ]:
# 保存模块
from tvm.contrib import utils

temp = utils.tempdir()
path_lib = temp.relpath("deploy_lib.tar")
lib.export_library(path_lib)
print(temp.listdir())

In [ ]:
# 重新加载模块
loaded_lib = tvm.runtime.load_module(path_lib)
input_data = tvm.nd.array(data)

module = graph_executor.GraphModule(loaded_lib["default"](dev))
module.run(data=input_data)
out_deploy = module.get_output(0).numpy()

# 打印输出的前十个元素
print(out_deploy.flatten()[0:10])

# 检查是否一致
tvm.testing.assert_allclose(out_deploy, out, atol=1e-5)